# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
import api_keys
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city = pd.read_csv("../output_data/cities.csv")
city.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Bagdarin,54.4333,113.6000,3.90,85,100,1.07,RU,1611469644
1,Bethel,41.3712,-73.4140,21.20,45,1,13.80,US,1611469850
2,Mataura,-46.1927,168.8643,57.99,78,53,18.99,NZ,1611469851
3,Maragogi,-9.0122,-35.2225,75.94,84,94,8.01,BR,1611469852
4,Bluff,-46.6000,168.3333,57.99,78,58,21.00,NZ,1611469675


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
#storing the lat & lng into locations
locations = city[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
humidity = city["Humidity"].astype(float)

# Create a Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the dataframe to find the ideal weather conditions (a temp lower than 75 but higher than 70)
ideal_weather_df = city.loc[(city["Max_Temp"] < 75) & (city["Max_Temp"] > 70)]

#wind speed less than 10 mph
ideal_weather_df = ideal_weather_df.loc[(city["Wind_Speed"] < 10)]

#0 cloudiness
ideal_weather_df = ideal_weather_df.loc[(city["Cloudiness"] == 0)]

#drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
108,Foz do Iguaçu,-25.5478,-54.5881,73.40,100,0,4.61,BR,1611469941
212,Mar del Plata,-38.0023,-57.5575,73.00,94,0,6.91,AR,1611470027
227,San Juan del Paraná,-27.3000,-55.9667,72.00,83,0,4.05,PY,1611470042
270,General Roca,-39.0333,-67.5833,73.40,46,0,6.91,AR,1611470083
334,Nueve de Julio,-35.4444,-60.8831,73.00,64,0,3.00,AR,1611470143
346,Bloemfontein,-29.1211,26.2140,73.40,68,0,4.61,ZA,1611470154
357,Itakyry,-24.9333,-55.2167,73.40,100,0,4.61,PY,1611470164
396,São João da Barra,-21.6403,-41.0511,72.37,87,0,5.21,BR,1611470200
405,Neuquén,-38.9516,-68.0591,73.40,46,0,6.91,AR,1611470209
416,Buin,-33.7333,-70.7500,72.00,51,0,1.05,CL,1611470220


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# adding the hotel name
hotel_df = ideal_weather_df.loc[:,["City","Country","Lat","Lng"]]

#adding the empty list for hotel name
hotel_df["Hotel name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel name
108,Foz do Iguaçu,BR,-25.5478,-54.5881,
212,Mar del Plata,AR,-38.0023,-57.5575,
227,San Juan del Paraná,PY,-27.3000,-55.9667,
270,General Roca,AR,-39.0333,-67.5833,
334,Nueve de Julio,AR,-35.4444,-60.8831,
346,Bloemfontein,ZA,-29.1211,26.2140,
357,Itakyry,PY,-24.9333,-55.2167,
396,São João da Barra,BR,-21.6403,-41.0511,
405,Neuquén,AR,-38.9516,-68.0591,
416,Buin,CL,-33.7333,-70.7500,


In [7]:
#create a params
params = {"type" : "lodging",
          "radius" : 5000,
          "key" : g_key}

#looping through the hotel_df and run lat/lng for each city
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # Update location key value 
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    
    # Save the hotel name to hotel_df
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # If there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Delay the code to avoid an error
    time.sleep(1)

# Print end of search once searching is completed
print("End of Search")

Retrieving Results for Index 108: Foz do Iguaçu.
Closest hotel in Foz do Iguaçu is Turrance Green Hotel.
------------
Retrieving Results for Index 212: Mar del Plata.
Closest hotel in Mar del Plata is Gran Hotel Mar del Plata.
------------
Retrieving Results for Index 227: San Juan del Paraná.
Closest hotel in San Juan del Paraná is Del Sur Hotel - Museo.
------------
Retrieving Results for Index 270: General Roca.
Closest hotel in General Roca is Hotel El Recreo.
------------
Retrieving Results for Index 334: Nueve de Julio.
Closest hotel in Nueve de Julio is Grand Hotel Libertad - GHL.
------------
Retrieving Results for Index 346: Bloemfontein.
Closest hotel in Bloemfontein is Protea Hotel by Marriott Bloemfontein.
------------
Retrieving Results for Index 357: Itakyry.
Closest hotel in Itakyry is Posada Santa Celina.
------------
Retrieving Results for Index 396: São João da Barra.
Closest hotel in São João da Barra is Pousada Porto De Canoas.
------------
Retrieving Results for In

In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel name,Hotel Name
108,Foz do Iguaçu,BR,-25.5478,-54.5881,,Turrance Green Hotel
212,Mar del Plata,AR,-38.0023,-57.5575,,Gran Hotel Mar del Plata
227,San Juan del Paraná,PY,-27.3000,-55.9667,,Del Sur Hotel - Museo
270,General Roca,AR,-39.0333,-67.5833,,Hotel El Recreo
334,Nueve de Julio,AR,-35.4444,-60.8831,,Grand Hotel Libertad - GHL
346,Bloemfontein,ZA,-29.1211,26.2140,,Protea Hotel by Marriott Bloemfontein
357,Itakyry,PY,-24.9333,-55.2167,,Posada Santa Celina
396,São João da Barra,BR,-21.6403,-41.0511,,Pousada Porto De Canoas
405,Neuquén,AR,-38.9516,-68.0591,,Hotel del Comahue
416,Buin,CL,-33.7333,-70.7500,,Paihuen


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))